## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.142"

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")

cursor = conn.cursor()


## Code to Add All Subdirs of "Python_scripts" to sys.path

In [ ]:
import sys
from pathlib import Path

# Add Python_scripts to sys.path (parent of Extract_db_columns)
scripts_dir = Path().resolve().parents[0] / "Python_scripts"
sys.path.append(str(scripts_dir))


In [ ]:
# cursor.execute("""
#     DELETE FROM dlc_table
#     WHERE head_x IS NULL AND genotype IS NULL
# """)
# conn.commit()

# cursor.execute("""
#     UPDATE dlc_table
#     SET task = 'ToyLightNew'
#     WHERE task = 'ToyLight';
# """)
# conn.commit()


## Insert video file names (Run this from the computer where the videos are located)

In [ ]:
from pathlib import Path

# Get the root of the project (i.e., where Jupyter Lab started)
project_root = Path().resolve().parents[0]

# Define base folder relative to root
# base_folder = project_root / "data" / "ToyOnly" / "SplitVideos"
base_folder = Path.home() / "Downloads" / "data" / "ToyLight" / "SplitVideos"

# Check path
print(f"Looking in: {base_folder}")
print(f"Exists? {base_folder.exists()}")

# Collect video files
video_files = list(base_folder.rglob("*.mp4")) + list(base_folder.rglob("*.avi"))

# Convert to relative paths (relative to 'data')
video_records = [(str(vf.relative_to(base_folder)),) for vf in video_files]

# Insert into dlc_table
insert_query = "INSERT INTO dlc_table (video_name) VALUES (%s);"

# Avoid Duplicate Inserts
cursor.execute("SELECT video_name FROM dlc_table;")
existing = set(row[0] for row in cursor.fetchall())

# Filter only new videos
new_records = [vr for vr in video_records if vr[0] not in existing]

cursor.executemany(insert_query, new_records)
conn.commit()


## Insert task, date_str, name, health, id_ in dlc_table

In [ ]:
import importlib
import Extract_db_columns.parse_video_name
importlib.reload(Extract_db_columns.parse_video_name)
from Extract_db_columns.parse_video_name import parse_video_name

cursor.execute("SELECT id, video_name FROM dlc_table WHERE video_name IS NOT NULL AND task IS NULL;")
rows = cursor.fetchall()

updates = []

for row in rows:
    id_, video_name = row
    task, date_str, name, health = parse_video_name(video_name)
    updates.append((task, date_str, name, health, id_))

# Update in batch
cursor.executemany("""
UPDATE dlc_table
SET task = %s, date = %s, name = %s, health = %s
WHERE id = %s;
""", updates)

conn.commit()


## Insert num_frames, frame_rate, video_width, video_height

In [ ]:
import importlib
import Extract_db_columns.update_video_info

importlib.reload(Extract_db_columns.update_video_info)

# Call the function with subdirectories
video_subdirs = ['ToyLight'] # ['FoodLight', 'FoodOnly', 'LightOnly', 'ToyOnly', 'ToyLight']
Extract_db_columns.update_video_info.update_video_info_in_db(video_subdirs)


## Insert raw coordinates for bodyparts

## Insert trial_length

In [ ]:
import importlib
import Insert_to_featuretable.insert_trial_length

importlib.reload(Insert_to_featuretable.insert_trial_length)
from Insert_to_featuretable.insert_trial_length import insert_trial_length

query = "SELECT id FROM dlc_table WHERE t IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)

id_list = df['id'].to_list()

insert_trial_length(id_list, conn)


## Insert genotype

In [ ]:
cursor = conn.cursor()
cursor.execute("UPDATE dlc_table SET genotype = 'black' WHERE genotype is NULL")
conn.commit()


### Completely Clear Table First (CAREFUL)

In [ ]:
cursor.execute("DELETE FROM dlc_files")
conn.commit()
print("All entries deleted from dlc_files.")
# conn.rollback()